In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
img_height = 28
img_width = 28
batch_size = 1

model = keras.Sequential([
    layers.Input((28, 28, 1)),
    layers.Conv2D(16, 3, padding ='same'),
    layers.Conv2D(32, 3, padding ='same'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(31),
])

model = Sequential()


In [3]:
ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset/train",
    #labels="inferred",
    label_mode="int",  # categorical, binary
    #class_names=['0', '1', '2', '3'],
    color_mode="grayscale",
    batch_size=batch_size,
    image_size=(img_height, img_width),  # reshape if not in this size
    shuffle=True,
    seed=123,
    validation_split=0.1,
    subset="training",
)

ds_validation = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset/valid",
    #labels="inferred",
    label_mode="int",  # categorical, binary
    #class_names=['0', '1', '2', '3', ...],
    color_mode="grayscale",
    batch_size=batch_size,
    image_size=(img_height, img_width),  # reshape if not in this size
    shuffle=True,
    seed=123,
    validation_split=0.1,
    subset="validation",
)


def augment(x, y):
    image = tf.image.random_brightness(x, max_delta=0.05)
    return image, y


ds_train = ds_train.map(augment)

# Custom Loops
for epochs in range(10):
    for x, y in ds_train:
        # train here
        pass


model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=[keras.losses.SparseCategoricalCrossentropy(from_logits=True),],
    metrics=["accuracy"],
)

model.fit(ds_train, epochs=200, verbose=1)

Found 5949 files belonging to 31 classes.
Using 5355 files for training.
Found 744 files belonging to 31 classes.
Using 74 files for validation.
Epoch 1/200
3374/5355 [=================>............] - ETA: 6s - loss: 10.0376 - accuracy: 0.2167

KeyboardInterrupt: 

In [ ]:
model.save('model.h5')